# Projeto MVP Sprint 2: Banco de Dados

Simone Costa Lírios Rocha

simone_jcc@yahoo.com.br


# Objetivo

O objetivo deste trabalho é analisar os dados populacionais e geográficos de todos os países do mundo para identificar padrões e tendências que possam informar políticas públicas, decisões de negócios, e estratégias de desenvolvimento sustentável. Especificamente, queremos entender a relação entre população, densidade populacional, taxa de crescimento populacional e outras métricas relevantes para avaliar o desenvolvimento de diferentes países.

A base de dados foi coletada do repositório de dados Kaggle, e a licensa de uso é de domínio público.
https://www.kaggle.com/datasets/dataanalyst001/world-population-by-country-2024

# Perguntas a serem respondidas

* Quais são os países com maior e menor população em 2024?
* Quais são os países com as maiores e menores densidades populacionais em 2024?
* Quais são os países com as maiores e menores taxas de crescimento populacional?
* Como a população de um país em 2024 está correlacionada com sua área geográfica?
* Como a densidade populacional está correlacionada com a taxa de crescimento populacional?

# Importação e Configuração

começamos importando as bibliotecas necessárias para a análise de dados: pandas para manipulação de dados tabulares em Python e pyspark para processamento de grandes volumes de dados distribuídos usando Spark. Em seguida, criamos uma sessão Spark com o nome "World Population Analysis", configurada para incluir o conector spark-bigquery, que permite interagir com o Google BigQuery. Este conector é especificado através da opção config ao inicializar o SparkSession. Finalmente, definimos o caminho do dataset wpc_2024.csv no Databricks File System (DBFS), um sistema de arquivos distribuído gerenciado pelo Databricks, onde o arquivo CSV está armazenado e será carregado para análise subsequente.

In [0]:
# Importar bibliotecas necessárias
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Criar uma sessão Spark
spark = SparkSession.builder \
    .appName("World Population Analysis") \
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.21.1") \
    .getOrCreate()

# Definir o caminho do dataset no Databricks File System (DBFS)
dbfs_path = "/FileStore/tables/wpc_2024.csv"


# Coleta dos Dados

Utiliza-se a funcionalidade de leitura do Spark para carregar o dataset armazenado no Databricks File System (DBFS). A função spark.read.csv é usada para ler o arquivo CSV localizado no caminho especificado por dbfs_path. Configuramos os parâmetros header=True para indicar que a primeira linha do CSV contém os nomes das colunas e inferSchema=True para que o Spark automaticamente deduza os tipos de dados de cada coluna. Após carregar o dataset em um DataFrame Spark chamado df, utilizamos o método show() para exibir os primeiros registros do dataset, permitindo uma visualização rápida dos dados carregados e a verificação de sua estrutura.

In [0]:
# Carregar o dataset do DBFS
df = spark.read.csv(dbfs_path, header=True, inferSchema=True)

# Mostrar os primeiros registros do dataset
df.show()


+-------------+---------------+---------------+----------+--------------+-----------+-------+----------+
|      Country|Population 2024|Population 2023|Area (km2)|Density (/km2)|Growth Rate|World %|World Rank|
+-------------+---------------+---------------+----------+--------------+-----------+-------+----------+
|        India|     1441719852|     1428627663|        3M|         485.0|     0.0092| 0.1801|         1|
|        China|     1425178782|     1425671352|      9.4M|         151.0|    -3.0E-4|  0.178|         2|
|United States|      341814420|      339996563|      9.1M|          37.0|     0.0053| 0.0427|         3|
|    Indonesia|      279798049|      277534122|      1.9M|         149.0|     0.0082|  0.035|         4|
|     Pakistan|      245209815|      240485658|    770.9K|         318.0|     0.0196| 0.0306|         5|
|      Nigeria|      229152217|      223804632|    910.8K|         252.0|     0.0239| 0.0286|         6|
|       Brazil|      217637297|      216422446|      8.

# Limpeza e Transformação dos Dados

Renomeia-se as colunas do DataFrame Spark df para facilitar o uso subsequente, simplificando os nomes das colunas originais para versões em letras minúsculas e sem espaços. Em seguida, convertemos explicitamente os tipos de dados das colunas relevantes: population_2024 e population_2023 são convertidas para inteiros (int), area_km2, density_per_km2, growth_rate e world_percent para ponto flutuante (float), e world_rank para inteiro (int). Realizamos uma verificação de valores ausentes em todas as colunas usando a função isNull e exibimos os resultados. Finalmente, removemos qualquer linha que contenha valores ausentes com o método dropna, assegurando que o DataFrame resultante esteja livre de dados incompletos e pronto para análise.

In [0]:
# Renomear colunas para facilitar o uso
df = df.withColumnRenamed("Country", "country") \
       .withColumnRenamed("Population 2024", "population_2024") \
       .withColumnRenamed("Population 2023", "population_2023") \
       .withColumnRenamed("Area (km2)", "area_km2") \
       .withColumnRenamed("Density (/km2)", "density_per_km2") \
       .withColumnRenamed("Growth Rate", "growth_rate") \
       .withColumnRenamed("World %", "world_percent") \
       .withColumnRenamed("World Rank", "world_rank")

# Converter tipos de dados
df = df.withColumn("population_2024", col("population_2024").cast("int")) \
       .withColumn("population_2023", col("population_2023").cast("int")) \
       .withColumn("area_km2", col("area_km2").cast("float")) \
       .withColumn("density_per_km2", col("density_per_km2").cast("float")) \
       .withColumn("growth_rate", col("growth_rate").cast("float")) \
       .withColumn("world_percent", col("world_percent").cast("float")) \
       .withColumn("world_rank", col("world_rank").cast("int"))

# Verificar se há valores ausentes
df.select([col(c).isNull().alias(c) for c in df.columns]).show()

# Remover linhas com valores ausentes
df = df.dropna()


+-------+---------------+---------------+--------+---------------+-----------+-------------+----------+
|country|population_2024|population_2023|area_km2|density_per_km2|growth_rate|world_percent|world_rank|
+-------+---------------+---------------+--------+---------------+-----------+-------------+----------+
|  false|          false|          false|    true|          false|      false|        false|     false|
|  false|          false|          false|    true|          false|      false|        false|     false|
|  false|          false|          false|    true|          false|      false|        false|     false|
|  false|          false|          false|    true|          false|      false|        false|     false|
|  false|          false|          false|    true|          false|      false|        false|     false|
|  false|          false|          false|    true|          false|      false|        false|     false|
|  false|          false|          false|    true|          fals

# Carga dos Dados no Delta Lake do Databricks

Os dados são salvos e processados em um formato Delta Lake, que é uma camada de armazenamento otimizada para operações de leitura e escrita no Databricks. Especificamos o caminho onde a tabela Delta será armazenada (delta_table_path) e utilizamos o método write do DataFrame para salvar os dados nesse local, com o formato delta e o modo overwrite para garantir que qualquer dado existente no caminho seja substituído. Após salvar os dados, criamos uma tabela Delta no catálogo do Spark SQL, usando uma consulta SQL. Esta tabela, chamada wpc_2024, é criada apenas se ainda não existir, e aponta para o caminho de armazenamento especificado. Isso permite que os dados sejam facilmente acessados e consultados via SQL, combinando a eficiência do Delta Lake com a flexibilidade das consultas SQL no Spark.

In [0]:
# Salvar os dados em formato Delta Lake
delta_table_path = "/mnt/delta/wpc_2024"
df.write.format("delta").mode("overwrite").save(delta_table_path)

# Criar uma tabela Delta
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS wpc_2024
    USING DELTA
    LOCATION '{delta_table_path}'
""")


Out[6]: DataFrame[]

# Análise dos Dados com Consultas SQL

Quais são os países com maior e menor população em 2024?

In [0]:
# Definir a consulta SQL
query_max_population = """
SELECT country, population_2024
FROM wpc_2024
ORDER BY population_2024 DESC
LIMIT 10
"""

query_min_population = """
SELECT country, population_2024
FROM wpc_2024
ORDER BY population_2024 ASC
LIMIT 10
"""

# Executar a consulta e exibir os resultados
df_pop_max = spark.sql(query_max_population)
df_pop_min = spark.sql(query_min_population)

df_pop_max.show()
df_pop_min.show()


+--------------------+---------------+
|             country|population_2024|
+--------------------+---------------+
|           Singapore|        6052709|
|             Bahrain|        1498712|
|               Macau|         713082|
|               Malta|         536740|
|            Maldives|         517887|
|             Mayotte|         345996|
|            Barbados|         282309|
|Sao Tome and Prin...|         236381|
|             Curacao|         192965|
|         Saint Lucia|         180805|
+--------------------+---------------+

+--------------------+---------------+
|             country|population_2024|
+--------------------+---------------+
|    Saint Barthelemy|          11019|
|   Wallis and Futuna|          11439|
|              Tuvalu|          11478|
|               Nauru|          12884|
|            Anguilla|          15936|
|        Cook Islands|          17072|
|               Palau|          18051|
|British Virgin Is...|          31763|
|        Saint Martin|  

Quais são os países com as maiores e menores densidades populacionais em 2024?

In [0]:
# Definir a consulta SQL
query_max_density = """
SELECT country, density_per_km2
FROM wpc_2024
ORDER BY density_per_km2 DESC
LIMIT 10
"""

query_min_density = """
SELECT country, density_per_km2
FROM wpc_2024
ORDER BY density_per_km2 ASC
LIMIT 10
"""

# Executar a consulta e exibir os resultados
df_density_max = spark.sql(query_max_density)
df_density_min = spark.sql(query_min_density)

df_density_max.show()
df_density_min.show()


+------------+---------------+
|     country|density_per_km2|
+------------+---------------+
|       Macau|        21674.0|
|      Monaco|        18079.0|
|   Singapore|         8430.0|
|   Gibraltar|         4811.0|
|     Bahrain|         1909.0|
|    Maldives|         1726.0|
|       Malta|         1677.0|
|Sint Maarten|         1303.0|
|     Bermuda|         1184.0|
|    Guernsey|         1013.0|
+------------+---------------+

+--------------------+---------------+
|             country|density_per_km2|
+--------------------+---------------+
|               Palau|           39.0|
|   Wallis and Futuna|           42.0|
|Turks and Caicos ...|           49.0|
|        Cook Islands|           71.0|
|            Dominica|           98.0|
|Northern Mariana ...|          109.0|
|         Isle of Man|          149.0|
|               Tonga|          151.0|
|          Micronesia|          166.0|
|            Kiribati|          168.0|
+--------------------+---------------+



Quais são os países com as maiores e menores taxas de crescimento populacional?

In [0]:
# Definir a consulta SQL
query_max_growth = """
SELECT country, growth_rate
FROM wpc_2024
ORDER BY growth_rate DESC
LIMIT 10
"""

query_min_growth = """
SELECT country, growth_rate
FROM wpc_2024
ORDER BY growth_rate ASC
LIMIT 10
"""

# Executar a consulta e exibir os resultados
df_growth_max = spark.sql(query_max_growth)
df_growth_min = spark.sql(query_min_growth)

df_growth_max.show()
df_growth_min.show()


+--------------------+-----------+
|             country|growth_rate|
+--------------------+-----------+
|             Mayotte|     0.0298|
|Sao Tome and Prin...|     0.0195|
|            Kiribati|     0.0168|
|               Macau|     0.0127|
|    Marshall Islands|       0.01|
|          Micronesia|     0.0093|
|              Jersey|     0.0093|
|             Bahrain|     0.0089|
|               Tonga|     0.0084|
|      Cayman Islands|     0.0083|
+--------------------+-----------+

+--------------------+-----------+
|             country|growth_rate|
+--------------------+-----------+
|      American Samoa|    -0.0084|
|United States Vir...|     -0.007|
|            Maldives|     -0.006|
|   Wallis and Futuna|    -0.0055|
|              Monaco|    -0.0039|
|             Bermuda|    -0.0021|
|               Aruba|    -9.0E-4|
|          San Marino|    -8.0E-4|
|               Palau|    -4.0E-4|
|Saint Vincent and...|    -1.0E-4|
+--------------------+-----------+



Como a população de um país em 2024 está correlacionada com sua área geográfica?

In [0]:
# Definir a consulta SQL
query_population_area = """
SELECT population_2024, area_km2
FROM wpc_2024
"""

# Executar a consulta e exibir os resultados
df_population_area = spark.sql(query_population_area)

# Calcular a correlação
correlation = df_population_area.corr("population_2024", "area_km2")
print(f"Correlação entre População em 2024 e Área Geográfica: {correlation}")


Correlação entre População em 2024 e Área Geográfica: 0.27302832470193183


Como a densidade populacional está correlacionada com a taxa de crescimento populacional?

In [0]:
# Definir a consulta SQL
query_density_growth = """
SELECT density_per_km2, growth_rate
FROM wpc_2024
"""

# Executar a consulta e exibir os resultados
df_density_growth = spark.sql(query_density_growth)

# Calcular a correlação
correlation = df_density_growth.corr("density_per_km2", "growth_rate")
print(f"Correlação entre Densidade Populacional e Taxa de Crescimento: {correlation}")


Correlação entre Densidade Populacional e Taxa de Crescimento: 0.012724154876021312


# Conclusão

Neste projeto, realizamos uma análise detalhada dos dados populacionais mundiais de 2024 utilizando a plataforma Databricks e Delta Lake. Começamos com a definição dos objetivos e das perguntas de interesse, coletamos e carregamos os dados no Databricks File System (DBFS), e realizamos a limpeza e transformação dos dados. Armazenamos o dataset processado em uma tabela Delta Lake, facilitando consultas SQL para responder perguntas sobre populações, densidades populacionais e taxas de crescimento dos países. As análises revelaram padrões e correlações significativas entre população, área geográfica e densidade populacional.